In [ ]:
import pygame
import sys
pygame.init()
screen = pygame.display.set_mode((600, 800))
pygame.display.set_caption("벽돌 깨기")
clock = pygame.time.Clock()

# --- 게임 변수 ---
# 공
ball = pygame.Rect(300, 400, 20, 20)
ball_speed = [5, -5]

# 발판
floor = pygame.Rect(250, 650, 100, 10)

# 블럭
bricks = []
bricks_color = (0, 255, 0)
for row in range(5):
    for col in range(10):
        brick = pygame.Rect(60 + col * 50, 40 + row * 30, 40, 20)
        bricks.append(brick)

# 점수
score = 0
game_font = pygame.font.SysFont(None, 36) # 점수 표시용 폰트
message_font = pygame.font.SysFont(None, 60) # 게임 종료 메시지용 폰트

def show_message(text, final_score):
    # 게임 종료 메시지에 최종 점수 포함
    message_text = f"{text} - Score: {final_score}"
    msg = message_font.render(message_text, True, (255, 0, 0))
    rect = msg.get_rect(center=(300, 400))
    screen.blit(msg, rect)
    pygame.display.update()
    pygame.time.delay(2000)

running = True
game_over_flag = False # 게임 오버/승리 상태 플래그

while running:
    screen.fill((0, 0, 0))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    if not game_over_flag: # 게임이 진행 중일 때만 아래 로직 실행
        # 패들 움직임
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and floor.left > 0:
            floor.x -= 7
        if keys[pygame.K_RIGHT] and floor.right < 600:
            floor.x += 7
            
        # 공 이동
        ball.x += ball_speed[0]
        ball.y += ball_speed[1]

        # 벽 충돌
        if ball.left <= 0 or ball.right >= 600:
            ball_speed[0] *= -1
        if ball.top <= 0:
            ball_speed[1] *= -1

        # 바닥 충돌
        if ball.colliderect(floor):
            ball_speed[1] *= -1
            # 바닥에 닿을 때 공이 발판 안으로 들어가는 것 방지
            if ball_speed[1] > 0: # 공이 아래로 향하고 있었다면 (이제 위로 튕겨나갈 것)
                ball.bottom = floor.top


        # 블록 충돌 체크 및 제거
        hit_brick_this_frame = False # 이번 프레임에 블록 충돌 여부
        for brick in bricks[:]: # 복사본을 순회하여 리스트 변경 문제 방지
            if ball.colliderect(brick):
                bricks.remove(brick)
                score += 10  # 블록 하나당 10점 추가
                hit_brick_this_frame = True

                # 공의 어느 면이 블록에 부딪혔는지 판단하여 반사 방향 결정
                # (간단한 반사 로직: 주로 y방향 반사, x방향 반사는 더 복잡한 로직 필요)

                # 공의 중심과 벽돌의 중심을 비교하여 충돌 방향 추정
                ball_center_x = ball.centerx
                ball_center_y = ball.centery
                brick_center_x = brick.centerx
                brick_center_y = brick.centery

                dx = ball_center_x - brick_center_x
                dy = ball_center_y - brick_center_y

                # 공이 벽돌의 옆면에 부딪혔는지, 윗면/아랫면에 부딪혔는지 판단
                # 벽돌의 너비와 높이에 대한 상대적인 위치를 고려
                width_collision = abs(dx) / (ball.width / 2 + brick.width / 2)
                height_collision = abs(dy) / (ball.height / 2 + brick.height / 2)

                if width_collision > height_collision: # 옆면 충돌
                    ball_speed[0] *= -1
                    # 공이 벽돌 안으로 파고드는 것 방지
                    if ball_speed[0] > 0: # 오른쪽으로 이동 중이었다면
                        ball.left = brick.right
                    else: # 왼쪽으로 이동 중이었다면
                        ball.right = brick.left
                else: # 윗면 또는 아랫면 충돌
                    ball_speed[1] *= -1
                     # 공이 벽돌 안으로 파고드는 것 방지
                    if ball_speed[1] > 0: # 아래로 이동 중이었다면
                        ball.top = brick.bottom
                    else: # 위로 이동 중이었다면
                        ball.bottom = brick.top
                
                break  # 한 번에 한 블록만 제거 및 충돌 처리

        # 블록 다 깼을 때
        if not bricks:
            show_message("You Win!", score)
            game_over_flag = True # 게임 종료 상태로 변경

        # 공이 아래로 떨어졌을 때
        if ball.top > 800:
            show_message("Game Over", score)
            game_over_flag = True # 게임 종료 상태로 변경

    # --- 그리기 ---
    # 발판
    pygame.draw.rect(screen, (255, 255, 255), floor)
    # 공
    pygame.draw.rect(screen, (123, 123, 123), ball)
    # 블록
    for brick in bricks:
        pygame.draw.rect(screen, bricks_color, brick)

    # 점수 표시 (게임 위쪽)
    score_text = game_font.render(f"Score: {score}", True, (255, 255, 255))
    screen.blit(score_text, (10, 10)) # 화면 좌측 상단에 표시

    pygame.display.update()
    
    if game_over_flag: # 게임이 종료되었으면 잠시 후 게임 종료
        pygame.time.delay(100) # show_message에서 이미 2초 딜레이가 있으므로 짧게 주거나 생략 가능
        running = False

    clock.tick(60)

pygame.quit()
sys.exit()

SystemExit: 